In [1]:
import func_mod as funciones  ###archivo de funciones propias
import pandas as pd ### para manejo de datos
import joblib
import openpyxl ## para exportar a excel
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import pandas as pd

In [5]:
df = pd.read_excel('despliegue//base_calificar.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'despliegue//base_calificar.xlsx'

In [10]:
df

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,InfoDate,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,6,1,0,0,2016-12-31,3,3,3.0,4.0,2.0
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,4412,Female,1,...,3,5,1,4,2016-12-31,2,4,3.0,2.0,4.0
2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,2,5,0,3,2016-12-31,3,3,2.0,2.0,1.0
3,38,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,...,5,8,7,5,2016-12-31,2,3,4.0,4.0,3.0
4,32,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,...,2,6,0,4,2016-12-31,3,3,4.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,Travel_Rarely,Research & Development,5,4,Medical,1,4406,Female,1,...,5,3,0,2,2016-12-31,3,3,4.0,1.0,3.0
4406,29,Travel_Rarely,Research & Development,2,4,Medical,1,4407,Male,1,...,2,3,0,2,2016-12-31,2,3,4.0,4.0,3.0
4407,25,Travel_Rarely,Research & Development,25,2,Life Sciences,1,4408,Male,2,...,4,4,1,2,2016-12-31,3,4,1.0,3.0,3.0
4408,42,Travel_Rarely,Sales,18,2,Medical,1,4409,Male,1,...,2,9,7,8,2016-12-31,2,3,4.0,1.0,3.0


In [11]:
df['EmployeeID'] = df['EmployeeID'].astype(str)
df.drop('EmployeeCount', axis=1, inplace=True)
df.drop('StandardHours', axis=1, inplace=True)
df.drop('Over18', axis=1, inplace=True)
df.drop('InfoDate', axis=1, inplace=True)
list_dummies=['BusinessTravel', 'Department','EducationField','Gender', 'JobRole', 'MaritalStatus']
df=pd.get_dummies(df,columns=list_dummies)

ids = df['EmployeeID']

X= df.loc[:,~df.columns.isin(['EmployeeID'])]

In [13]:


# Cargar el modelo guardado
loaded_model = joblib.load('random_forest_model.pkl')

# Ahora puedes usar `loaded_model` para hacer predicciones
y_pred = loaded_model.predict(X)
y_scores = loaded_model.predict_proba(X)[:, 1]

In [16]:
# Crear un DataFrame con las predicciones y las probabilidades
predictions_df = pd.DataFrame({
    'id_empleado': ids,
    'predicted_probabilities': y_scores  # Probabilidades
})
# Definir los rangos de las probabilidades (0 a 1 en intervalos de 0.1)
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Etiquetas para los intervalos
labels = ['0-0.1', '0.1-0.2', '0.2-0.3', '0.3-0.4', '0.4-0.5', '0.5-0.6', '0.6-0.7', '0.7-0.8', '0.8-0.9', '0.9-1.0']

# Crear una nueva columna que clasifique las probabilidades en rangos
predictions_df['probability_range'] = pd.cut(predictions_df['predicted_probabilities'], bins=bins, labels=labels, include_lowest=True)

# Contar cuántas predicciones caen en cada rango
count_by_range = predictions_df['probability_range'].value_counts().sort_index()
# Definir los rangos de las probabilidades
bins = [0, 0.3, 0.7, 1.0]

# Etiquetas para los niveles de riesgo
labels = ['Riesgo bajo', 'Riesgo medio', 'Riesgo alto']

# Crear la nueva columna 'riesgo_desercion' con base en los rangos de probabilidad
predictions_df['riesgo_desercion'] = pd.cut(predictions_df['predicted_probabilities'], bins=bins, labels=labels, include_lowest=True)

In [17]:
df_final = pd.DataFrame()
df_final["Prob_desercion"]= (round(predictions_df['predicted_probabilities'] * 100,2)).astype(str) + '%'
df_final["Riesgo_desercion"] = predictions_df["riesgo_desercion"]

In [18]:
df_final

,Prob_desercion,Riesgo_desercion
0,15.0%,Riesgo bajo
1,97.33%,Riesgo alto
2,2.68%,Riesgo bajo
3,1.38%,Riesgo bajo
4,0.0%,Riesgo bajo
...,...,...
4405,0.44%,Riesgo bajo
4406,0.0%,Riesgo bajo
4407,1.4%,Riesgo bajo
4408,8.0%,Riesgo bajo


In [19]:
importances = loaded_model.feature_importances_
df_impor = pd.DataFrame({'Variable': X.columns, 'Importancia': importances})
df_impor = df_impor.sort_values(by='Importancia', ascending=False)

In [3]:
if __name__=="__main__":
    
    df = pd.read_excel(r'C:\Users\alejo\OneDrive\Documentos\Analítica-3\Repos\Recursos_humanos_git\Proyecto-Analitica-3\data\base_calificar.xlsx')

    df['EmployeeID'] = df['EmployeeID'].astype(str)
    df.drop('EmployeeCount', axis=1, inplace=True)
    df.drop('StandardHours', axis=1, inplace=True)
    df.drop('Over18', axis=1, inplace=True)
    df.drop('InfoDate', axis=1, inplace=True)
    list_dummies=['BusinessTravel', 'Department','EducationField','Gender', 'JobRole', 'MaritalStatus']
    df=pd.get_dummies(df,columns=list_dummies)

    ids = df['EmployeeID']

    X= df.loc[:,~df.columns.isin(['EmployeeID'])]
    
    import joblib

    # Cargar el modelo guardado
    loaded_model = joblib.load('random_forest_model.pkl')

    # Ahora puedes usar `loaded_model` para hacer predicciones
    y_pred = loaded_model.predict(X)
    y_scores = loaded_model.predict_proba(X)[:, 1]

    # Crear un DataFrame con las predicciones y las probabilidades
    predictions_df = pd.DataFrame({
        'id_empleado': ids,
        'predicted_probabilities': y_scores  # Probabilidades
    })
    # Definir los rangos de las probabilidades (0 a 1 en intervalos de 0.1)
    bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    # Etiquetas para los intervalos
    labels = ['0-0.1', '0.1-0.2', '0.2-0.3', '0.3-0.4', '0.4-0.5', '0.5-0.6', '0.6-0.7', '0.7-0.8', '0.8-0.9', '0.9-1.0']

    # Crear una nueva columna que clasifique las probabilidades en rangos
    predictions_df['probability_range'] = pd.cut(predictions_df['predicted_probabilities'], bins=bins, labels=labels, include_lowest=True)

    # Contar cuántas predicciones caen en cada rango
    count_by_range = predictions_df['probability_range'].value_counts().sort_index()
    # Definir los rangos de las probabilidades
    bins = [0, 0.3, 0.7, 1.0]

    # Etiquetas para los niveles de riesgo
    labels = ['Riesgo bajo', 'Riesgo medio', 'Riesgo alto']

    # Crear la nueva columna 'riesgo_desercion' con base en los rangos de probabilidad
    predictions_df['riesgo_desercion'] = pd.cut(predictions_df['predicted_probabilities'], bins=bins, labels=labels, include_lowest=True)

    df_final = pd.DataFrame()
    df_final["Prob_desercion"]= (round(predictions_df['predicted_probabilities'] * 100,2)).astype(str) + '%'
    df_final["Riesgo_desercion"] = predictions_df["riesgo_desercion"]

    importances = loaded_model.feature_importances_
    df_impor = pd.DataFrame({'Variable': X.columns, 'Importancia': importances})
    df_impor = df_impor.sort_values(by='Importancia', ascending=False)
    
    df_final.to_excel(r"C:\Users\alejo\OneDrive\Documentos\Analítica-3\Repos\Recursos_humanos_git\Proyecto-Analitica-3\salidas\predicciones.xlsx")   #### exportar predicciones
    df_impor.to_excel(r"C:\Users\alejo\OneDrive\Documentos\Analítica-3\Repos\Recursos_humanos_git\Proyecto-Analitica-3\salidas\importancia_var.xlsx") ### exportar coeficientes para analizar predicciones